<p style="font-weight:bold;"> <span style="font-size: 36px"> Market updates on impact of IFRS 17 and IFRS 9 </span> </p>

<p style="font-weight:bold;"> <span style="font-size: 21px"> Updates on Shareholders equity for transition to the new accounting standard </span> </p>

# Accounting standards

## IFRS 4

// TODO: to be reviewed

IFRS 4 was introduced as an intermediary accounting standard between older established standards, primarily US GAAP, and the new standard IFRS 17.

On the liability side of the balance sheet, IFRS 4 allows provisions for future claims, benefits and expenses (and also expected premiums to be received in the future) to be accounted at nominal value, i.e. **undiscounted**, as in older standards.  IFRS 4 alternatively welcomes methods closer to a fair value accounting, i.e. discounting some expected future cash flows, which some insurers actually do to a certain extent.

The liabilities are accounted for in this sense, leading to a sum of liabilities which also includes and some smaller amounts, here summarized as "other liabilities".  Two main newly introduced liability concepts, the Contractual Service Margin (CSM) and the Risk Adjustment (RA), are unknown in IFRS 4, so they do not exist in the sum of liabilities.

The Shareholders' Equity is defined in the standard way:

**Shareholders' Equity  =  Total Assets  -  Sum of Liabilities** 

The presented example assumes undiscounted IFRS 4 liabilities for the sake of a clear comparison to IFRS 17.

In the next line the IFRS 4 business logic is imported:

In [0]:
#!import "Ifrs 4 Positions"

## IFRS 17

// TODO: @Ulri here pls place rules and explanations about the novel items and concepts that IFRS 17 brought to the insurance industry

A fundamental concept introduced by IFRS 17 is the **Contractual Service Margin** (**CSM**).  The basic idea is that profits should not figure in a balance sheet if they are expected to materialize after the as-of date of the balance sheet. Instead, these expected future profits constitute the CSM, a liability to be added to the sum of liabilities and thus reducing the Shareholders' Equity, following the definition

Shareholders' Equity  =  Total Assets  -  Sum of Liabilities

The other


and the Risk Adjustment (RA)

In the next line the IFRS 17 business logic is imported:

In [0]:
//#!import "//ifrs17/v1.1.1/CalculationEngine"
#!import "../../../ifrs17/CalculationEngine"

# Case Study

// TODO: @Ulri here pls place relevant info that would allow an unexperienced analyst to understand what to expect from the transition to the new standard with respect to the **equity**

We study ...

# FROM HERE ON ANDREA IS WORKING

## Data Import

### Dimension Initialization

In [0]:
#!eval-notebook "Dimensions Initialization"

### Data Nodes

In [0]:
await Import.FromFile("../../Files/DataNodes/DataNodes_CH.csv").WithFormat(ImportFormats.DataNode).WithTarget(DataSource).ExecuteAsync()

### Transactional Data

In [0]:
await Import.FromFile("../../Files/TransactionalData/NominalCashflows_CH_2020_12.csv").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromFile("../../Files/TransactionalData/Actuals_CH_2020_12.csv").WithFormat(ImportFormats.Actual).WithTarget(DataSource).WithActivityLog().ExecuteAsync()

### IFRS 4 Import 

In [0]:
await Import.FromFile("../../Files/TransactionalData/NominalCashflows_CH_2020_12.csv").WithFormat("IFRS 4 Cashflow").WithTarget(DataSource).ExecuteAsync()

## Results Analysis

In [0]:
var ReportingNode = "CH";
var Year = 2020;
var Month = 12; 
var Scenario = default(string);

In [0]:
var ifrs4Equity = (await Workspace.QueryReportVariablesAsync((Year, Month, ReportingNode, Scenario)))
    .Where(x => x.EstimateType == "IFRS4EQ" && x.VariableType == "BOP" && x.LiabilityType == "LRC").ToDataCube();

In [0]:
var ifrs17Equity = (await Workspace.QueryReportVariablesAsync((Year, Month, ReportingNode, Scenario)))
    .Where(x => x.EstimateType == "C" && x.VariableType == "AM" && x.LiabilityType == "LRC").Select(x => x with { EstimateType = "IFRS17EQ", Value = -x.Value }).ToDataCube();

In [0]:
var ifrs17CSM = (await Workspace.QueryReportVariablesAsync((Year, Month, ReportingNode, Scenario)))
    .Where(x => x.EstimateType == "C" && x.VariableType == "BOP" && x.LiabilityType == "LRC").Select(x => x with { EstimateType = "IFRS17CSM" }).ToDataCube();

In [0]:
var ifrs17EquityPlusCSM = ifrs17Equity.Select(x => x with { EstimateType = "IFRS17EQCSM" }).ToDataCube() 
                        + ifrs17CSM.Select(x => x with { EstimateType = "IFRS17EQCSM" }).ToDataCube();

In [0]:
var equity = ifrs4Equity + ifrs17Equity + ifrs17EquityPlusCSM;

In [0]:
(ifrs4Equity.Aggregate().Value, ifrs17Equity.Aggregate().Value, ifrs17EquityPlusCSM.Aggregate().Value)

### Reports

We can confirm with the current mock cash flows that, due to accounting for CSM, shareholder's equity will reduce.

However, the sum of Equity plus CSM is greater than Equity under IFRS 4. 

In [0]:
await Report.ForDataCube(equity)
            .WithQuerySource(DataSource)
            .SliceRowsBy(new string[]{})
            .SliceColumnsBy(new string[]{"EstimateType"})
            .ToTable()
            .WithOptions(x => x.WithColumns(cols => cols.Modify("Value", c => c.WithWidth(400))).WithAutoGroupColumn(c => c.WithWidth(400) with { Pinned = "left" }))
            .ExecuteAsync()

In [0]:
await Report.ForDataCube(equity)
            .WithQuerySource(DataSource)
            .SliceRowsBy(new string[]{})
            .SliceColumnsBy(new string[]{"EstimateType"})
            .ToBarChart()
//            .WithOptions(x => x.WithColumns(cols => cols.Modify("Value", c => c.WithWidth(400))).WithAutoGroupColumn(c => c.WithWidth(400) with { Pinned = "left" }))
            .ExecuteAsync()

TODO:

@Ulri: produce the tool in excel

few input figures

@Andrea: translate tool in SMAPP

<br/><br/><br/><br/><br/><br/><br/><br/>

# Testing 

In [0]:
ifrs4Results
//.Select(x => x.ToIdentityString())
.Select(x => (x.VariableType, x.Novelty, x.LineOfBusiness, x.LiabilityType, x.GroupOfContract, x.AmountType, x.EconomicBasis, x.Currency, x.ContractualCurrency, x.Value));

In [0]:
ifrs17Results
//.Select(x => x.ToIdentityString())
.Select(x => (x.VariableType, x.Novelty, x.LineOfBusiness, x.LiabilityType, x.GroupOfContract, x.AmountType, x.EconomicBasis, x.Currency, x.ContractualCurrency, x.Value));

In [0]:
DataSource.Query<IfrsVariable>().Where(x => x.EstimateType == "IFRS4EQ" && x.AocType == "BOP");

In [0]:
DataSource.Query<IfrsVariable>().Where(x => x.EstimateType == "C");

In [0]:
//var pv = ifrs17.PresentValues;
//pv.ReportingNode = "CH";
//pv.ReportingPeriod = (2020, 12);
//await pv.ToReportAsync

In [0]:
//await pv.GetStorage().InitializeAsync(pv.ReportingPeriod, pv.ReportingNode, (string)null, pv.CurrencyType);
//pv.GetScope<Data>((pv.ReportingPeriod, pv.ReportingNode, (string)null, pv.CurrencyType, nameof(PvReport), Enumerable.Empty<(string, object)>().ToArray())).Cube

In [0]:
//pv.GetScope<IIfrs17Report>(nameof(PvReport)).GetDataCube()

In [0]:
var technicalMargins = ifrs17.TechnicalMargins;
technicalMargins.ReportingNode = "CH";
technicalMargins.ReportingPeriod = (2020, 12);
technicalMargins.ColumnSlices = new string[]{};                                    //"GroupOfContract", "AmountType"
technicalMargins.DataFilter = new [] {("GroupOfContract", "DT1.1")};               //new [] {("GroupOfContract", "DT1.1")};
await technicalMargins.ToReportAsync

In [0]:
var financialPerformance = ifrs17.FinancialPerformance;
financialPerformance.ReportingNode = "CH";
financialPerformance.ReportingPeriod = (2020, 12);
financialPerformance.ColumnSlices = new string[]{};                                 //"GroupOfContract"
financialPerformance.DataFilter = new [] {("GroupOfContract", "DT1.1")};            //new [] {("GroupOfContract", "DT1.1")};
(await financialPerformance.ToReportAsync) with {GroupDefaultExpanded = 3}